<h1 style="text-align:center;font-size:30px;" > Testing For Damage Prediction Modeling</h1>

<h1>1. Libraries</h1>

In [124]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import json
import pickle
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

<h2>1.1 Enable interactive widget in jupyter</h2>

In [125]:
# !pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


<h1>2 Fucntion to read variable details for prediction</h1>

In [126]:
def readType():

    with open('../variable-details/dataTypes.json', 'r') as f:
        data_types = json.load(f)
        
    with open('../variable-details/TrainDataTypes.json', 'r') as f:
        train_data_types = json.load(f)
        
    data_types.pop('Row_ID')
    data_types.pop('Household_ID')
    data_types.pop('Claim_Amount')

    same_col = []
    cols4dummies = ['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11', 'Cat12', 'OrdCat', 'NVCat']
    for i in data_types.keys():#list(data_types.keys())[0:5] + list(data_types.keys())[18:31]
        if i not in cols4dummies:
            same_col.append(i)
    return data_types, train_data_types, same_col



<h1>3 Fucntion to Load trained Model for Prediction</h1>

In [127]:


def loadModel():
    # load the model from disk
    loaded_model = pickle.load(open('../saved-model/RandomForestModel.sav', 'rb'))
    return loaded_model



<h1>4 Fucntion to Load Category Encoders saved as .pkl file</h1>

In [128]:
def encoderLoad():
    pickleLoaded = dict()
    for col in ['Blind_Make','Blind_Model','Blind_Submodel']:
        pkl_file = open('../encoder-files/'+col+'.pkl', 'rb')
        pick = pickle.load(pkl_file) 
        pickleLoaded[col] = pick
        pkl_file.close()
    return pickleLoaded



<h1>5 Function to Predict for a input data</h1>
<p>The function takes the 32 variables as inputs and then trasforms and ecnodes for prediction. The categoroial variables gets encodes and the object variable gets converted into one hot</p>

In [176]:
def predict(test_inpt, data_types, train_data_types, same_col):

    pickleLoaded = encoderLoad()
    loaded_model = loadModel()
    colm = list(train_data_types.keys())
    mod_inputs = pd.DataFrame(columns=colm)
    mod_inputs = mod_inputs.astype(train_data_types) 
    data = []
    
    for col in colm:
        if col in same_col:
            
            if col in ['Blind_Make','Blind_Model','Blind_Submodel']:
                le = pickleLoaded[col]
                test_inpt[col] = le.transform(test_inpt[col])
                data.append(test_inpt.loc[0,col])
            else:
                data.append(test_inpt.loc[0,col])
        else:
            data.append(0)

    s2 = pd.Series(data, index=list(train_data_types.keys()))
    mod_inputs = mod_inputs.append(s2,ignore_index=True)
    dummy_col = ['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11', 'Cat12', 'OrdCat', 'NVCat']
    for col in dummy_col:
        val = test_inpt.loc[0,col]
        val = col+"_"+val
        
        if val in list(mod_inputs.columns):
            
            mod_inputs.loc[0,val]=1
        #else:
        #    print(val)
        #print(val)
        #print(mod_inputs.shape)
    result = loaded_model.predict([mod_inputs.iloc[0,:]])
    return result[0]



<h1>6 Function to get a pandas dataframe of test inputs</h1>


In [179]:
def testInput(data, data_types):
    test_inpt = pd.DataFrame(columns=data_types.keys())
    test_inpt = test_inpt.astype(data_types) 

    s2 = pd.Series(data, index=list(data_types.keys()))
    test_inpt = test_inpt.append(s2,ignore_index=True)
    return test_inpt

    


<h1>7 The Raw data frame is loaded to populate drop down data for interactive buttons for testing data</h1>


In [172]:

data_types, train_data_types, same_col = readType()

df = pd.read_csv("../raw-data/podatki.csv")

print("Number of data points:",df.shape[0])
print("Number of data features:",df.shape[1])
df.drop(['Row_ID','Household_ID','Claim_Amount'], axis=1, inplace=True)

# data = []
# for i in range(len(data_types.keys())):
#     data.append(df.iloc[10880,i])
# test_inpt = testInput(data, data_types)

# t = predict(test_inpt, data_types, train_data_types, same_col)
# print("Predicted Result is",t)




Number of data points: 100000
Number of data features: 35


<h1>7 Test Using the below given interactive drop down module for prediction</h1>


In [182]:
@interact
def getResults(
 Vehicle=list(df['Vehicle'].unique()),
 Calendar_Year=list(df['Calendar_Year'].unique()),
 Model_Year=list(df['Model_Year'].unique()),
 Blind_Make=list(df['Blind_Make'].unique()),
 Blind_Model=list(df['Blind_Model'].unique()),
 Blind_Submodel=list(df['Blind_Submodel'].unique()),
 Cat1=list(df['Cat1'].unique()),
 Cat2=list(df['Cat2'].unique()),
 Cat3=list(df['Cat3'].unique()),
 Cat4=list(df['Cat4'].unique()),
 Cat5=list(df['Cat5'].unique()),
 Cat6=list(df['Cat6'].unique()),
 Cat7=list(df['Cat7'].unique()),
 Cat8=list(df['Cat8'].unique()),
 Cat9=list(df['Cat9'].unique()),
 Cat10=list(df['Cat10'].unique()),
 Cat11=list(df['Cat11'].unique()),
 Cat12=list(df['Cat12'].unique()),
 OrdCat=list(df['OrdCat'].unique()),
 Var1=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var2=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var3=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var4=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var5=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var6=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var7=widgets.FloatText(
    value=0.5,
    disabled=False
),
 Var8=widgets.FloatText(
    value=0.5,
    disabled=False
),
 NVCat=list(df['NVCat'].unique()),
 NVVar1=widgets.FloatText(
    value=0.5,
    disabled=False
),
 NVVar2=widgets.FloatText(
    value=0.5,
    disabled=False
),
 NVVar3=widgets.FloatText(
    value=0.5,
    disabled=False
),
 NVVar4=
widgets.FloatText(
    value=7.5,
    disabled=False
)

):
    data=[Vehicle,
 Calendar_Year,
 Model_Year,
 Blind_Make,
 Blind_Model,
 Blind_Submodel,
 Cat1,
 Cat2,
 Cat3,
 Cat4,
 Cat5,
 Cat6,
 Cat7,
 Cat8,
 Cat9,
 Cat10,
 Cat11,
 Cat12,
 OrdCat,
 Var1,
 Var2,
 Var3,
 Var4,
 Var5,
 Var6,
 Var7,
 Var8,
 NVCat,
 NVVar1,
 NVVar2,
 NVVar3,
 NVVar4]
    test_inpt = testInput(data, data_types)

    t = predict(test_inpt, data_types, train_data_types, same_col)
    print("Predicted Damage Result is",t)

interactive(children=(Dropdown(description='Vehicle', options=(4, 1, 3, 5, 2, 6, 8, 7, 9, 10, 12, 11, 14, 17, …